# Data Cleaning & Feature Engineering

## Objective
Prepare the housing dataset for machine learning by handling missing values,
fixing data types, and encoding categorical variables where required.

## Inputs
- Cleaned outputs from Phase 2
- house_prices_records.csv

## Outputs
- Cleaned dataset ready for modeling
- Clear justification for each data cleaning decision


## Missing Value Handling Strategy

Missing values in the dataset represent different real-world scenarios.

- Some missing values indicate the absence of a property feature (e.g. no garage or no basement)
- Some missing values represent unknown measurements and require statistical imputation

The following strategies are applied:

### Categorical Features
Missing values are filled with `"None"` to indicate absence of the feature.

### Numerical Features (Absence-based)
Missing values are filled with `0` where absence implies zero area or size.

### Numerical Features (Measurement-based)
Missing values are filled using the median to avoid distortion caused by outliers.

## Cleaning Outcome

All missing values have been handled using domain-informed strategies.
The dataset is now complete and suitable for feature engineering and modeling.

### Additional Missing Values Handling

- `BedroomAbvGr` missing values were filled using the median as this represents a measurement-based feature.
- `2ndFlrSF` missing values were filled with 0 to represent absence of a second floor.


## Feature Engineering Outcome

All categorical features were ordinally encoded based on domain knowledge.
The dataset now contains only numerical features and is fully prepared for modeling.


In [2]:
import pandas as pd

train_df = pd.read_csv("../data/raw/house_prices_records.csv")

train_df.shape


(1460, 24)

In [3]:
# Fill categorical missing values with 'None'
categorical_fill_none = [
    "BsmtExposure",
    "BsmtFinType1",
    "GarageFinish"
]

for col in categorical_fill_none:
    train_df[col] = train_df[col].fillna("None")

train_df[categorical_fill_none].isnull().sum()


BsmtExposure    0
BsmtFinType1    0
GarageFinish    0
dtype: int64

In [4]:
# Fill numerical features where NaN means absence
numerical_fill_zero = [
    "EnclosedPorch",
    "WoodDeckSF",
    "MasVnrArea"
]

for col in numerical_fill_zero:
    train_df[col] = train_df[col].fillna(0)

train_df[numerical_fill_zero].isnull().sum()


EnclosedPorch    0
WoodDeckSF       0
MasVnrArea       0
dtype: int64

In [5]:
# Fill important numerical features using median
median_fill = [
    "LotFrontage",
    "GarageYrBlt"
]

for col in median_fill:
    train_df[col] = train_df[col].fillna(train_df[col].median())

train_df[median_fill].isnull().sum()


LotFrontage    0
GarageYrBlt    0
dtype: int64

In [6]:
train_df.isnull().sum().sort_values(ascending=False)


BedroomAbvGr     99
2ndFlrSF         86
1stFlrSF          0
BsmtExposure      0
BsmtFinSF1        0
BsmtFinType1      0
BsmtUnfSF         0
EnclosedPorch     0
GarageArea        0
GarageFinish      0
GarageYrBlt       0
GrLivArea         0
KitchenQual       0
LotArea           0
LotFrontage       0
MasVnrArea        0
OpenPorchSF       0
OverallCond       0
OverallQual       0
TotalBsmtSF       0
WoodDeckSF        0
YearBuilt         0
YearRemodAdd      0
SalePrice         0
dtype: int64

In [7]:
# Handle remaining missing values

# BedroomAbvGr: measurement-based -> median
train_df["BedroomAbvGr"] = train_df["BedroomAbvGr"].fillna(
    train_df["BedroomAbvGr"].median()
)

# 2ndFlrSF: absence-based -> 0
train_df["2ndFlrSF"] = train_df["2ndFlrSF"].fillna(0)


In [8]:
train_df.isnull().sum().sort_values(ascending=False)


1stFlrSF         0
2ndFlrSF         0
BedroomAbvGr     0
BsmtExposure     0
BsmtFinSF1       0
BsmtFinType1     0
BsmtUnfSF        0
EnclosedPorch    0
GarageArea       0
GarageFinish     0
GarageYrBlt      0
GrLivArea        0
KitchenQual      0
LotArea          0
LotFrontage      0
MasVnrArea       0
OpenPorchSF      0
OverallCond      0
OverallQual      0
TotalBsmtSF      0
WoodDeckSF       0
YearBuilt        0
YearRemodAdd     0
SalePrice        0
dtype: int64

In [9]:
categorical_cols = train_df.select_dtypes(include=["object", "string"]).columns
categorical_cols



Index(['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'], dtype='str')

In [10]:
kitchen_map = {
    "Ex": 4,
    "Gd": 3,
    "TA": 2,
    "Fa": 1,
    "None": 0
}

train_df["KitchenQual"] = train_df["KitchenQual"].map(kitchen_map)

train_df["KitchenQual"].value_counts().sort_index()


KitchenQual
1     39
2    735
3    586
4    100
Name: count, dtype: int64

In [11]:
bsmt_exposure_map = {
    "Gd": 4,
    "Av": 3,
    "Mn": 2,
    "No": 1,
    "None": 0
}

train_df["BsmtExposure"] = train_df["BsmtExposure"].map(bsmt_exposure_map)

train_df["BsmtExposure"].value_counts().sort_index()


BsmtExposure
0     38
1    953
2    114
3    221
4    134
Name: count, dtype: int64

In [12]:
bsmt_fin_map = {
    "GLQ": 6,   # Good Living Quarters
    "ALQ": 5,   # Average Living Quarters
    "BLQ": 4,   # Below Average Living Quarters
    "Rec": 3,   # Average Rec Room
    "LwQ": 2,   # Low Quality
    "Unf": 1,   # Unfinished
    "None": 0   # No Basement
}

train_df["BsmtFinType1"] = train_df["BsmtFinType1"].map(bsmt_fin_map)

train_df["BsmtFinType1"].value_counts().sort_index()


BsmtFinType1
0    145
1    396
2     70
3    126
4    136
5    202
6    385
Name: count, dtype: int64

In [13]:
garage_finish_map = {
    "Fin": 3,   # Finished
    "RFn": 2,   # Rough Finished
    "Unf": 1,   # Unfinished
    "None": 0   # No Garage
}

train_df["GarageFinish"] = train_df["GarageFinish"].map(garage_finish_map)

train_df["GarageFinish"].value_counts().sort_index()


GarageFinish
0    235
1    546
2    366
3    313
Name: count, dtype: int64

In [14]:
train_df.select_dtypes(include=["object", "string"]).columns


Index([], dtype='str')

In [15]:
train_df.head()


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,1,706,6,150,0.0,548,2,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,4,978,5,284,0.0,460,2,...,80.0,0.0,0,8,6,1262,0.0,1976,1976,181500
2,920,866.0,3.0,2,486,6,434,0.0,608,2,...,68.0,162.0,42,5,7,920,0.0,2001,2002,223500
3,961,0.0,3.0,1,216,5,540,0.0,642,1,...,60.0,0.0,35,5,7,756,0.0,1915,1970,140000
4,1145,0.0,4.0,3,655,6,490,0.0,836,2,...,84.0,350.0,84,5,8,1145,0.0,2000,2000,250000


In [16]:
train_df.isnull().sum().sort_values(ascending=False)


1stFlrSF         0
2ndFlrSF         0
BedroomAbvGr     0
BsmtExposure     0
BsmtFinSF1       0
BsmtFinType1     0
BsmtUnfSF        0
EnclosedPorch    0
GarageArea       0
GarageFinish     0
GarageYrBlt      0
GrLivArea        0
KitchenQual      0
LotArea          0
LotFrontage      0
MasVnrArea       0
OpenPorchSF      0
OverallCond      0
OverallQual      0
TotalBsmtSF      0
WoodDeckSF       0
YearBuilt        0
YearRemodAdd     0
SalePrice        0
dtype: int64

In [17]:
import os

# create processed folder if not exists
os.makedirs("../data/processed", exist_ok=True)

# save cleaned dataset
train_df.to_csv("../data/processed/clean_train.csv", index=False)

print("✅ Clean dataset saved")
print("Shape:", train_df.shape)


✅ Clean dataset saved
Shape: (1460, 24)
